In [2]:
import pandas as pd
import re

posts = pd.read_csv('Cleaned Posts/PostsOnly-wallstreetbets-NOAutoMod.csv')
comments = pd.read_csv('Cleaned Comments/CommentsOnly-wallstreetbets-NOAutoMod.csv')

postsData = posts.copy()
postsData['title'] = posts['title'].fillna('').astype(str) + " " + posts['selftext'].fillna('').astype(str)



# bodyComments = comments['body']
# postsData = postsData['title']

# print(bodyComments.shape)
# print(postsData.shape)

# subData = postsData.append(bodyComments)

# print(subData.shape)


documentArray = []


linkIDs = comments['link_id'].unique().tolist()

counter = 0

for item in linkIDs:
    returnString = ""
    
    
    ###Get the comments of the specific post link_id
    commentsofPost = comments[comments['link_id'] == item].fillna('').astype(str)
    for index, row in commentsofPost.iterrows():
        #Remove URLs from strings
        singleComment = str(row['body'])
        removedURL = re.sub(r"http\S+", "", singleComment)
        returnString += removedURL + " "
    ################################################
    
    
    ###Get Post and post body of post link_id 
    postTitleBody = postsData[postsData['id'] == item].fillna('').astype(str)
    postString = str(postTitleBody.iloc[0]['title']) + " " + str(postTitleBody.iloc[0]['selftext'])
    postString = re.sub(r"http\S+", "", postString)
    returnString = postString + returnString
    ################################################

    documentArray.append(str(returnString))

#Convert to pandas
subDocuments = pd.DataFrame(documentArray)
subDocuments = subDocuments.squeeze()
print(subDocuments.shape)

(328,)


In [3]:
#Data Pre-processing

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np


import nltk
nltk.download('wordnet')



stemmer = PorterStemmer()


my_stop_words = STOPWORDS.union(set(['like']))

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        # if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
        if token not in my_stop_words and len(token) > 2:
            result.append(lemmatize_stemming(token))
    return result


# doc_sample = selfTextPosts.iloc[5]

# print('original document: ')
# words = []
# for word in doc_sample.split(' '):
#     words.append(word)
# print(words)
# print('\n\n tokenized and lemmatized document: ')
# print(preprocess(doc_sample))



processed_docs = subDocuments.map(preprocess)
print(processed_docs[:10])

C:\Users\Nachiappan\AppData\Local\Programs\Python\Python38\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Nachiappan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
0    [hell, headlin, give, massiv, sticki, growth, ...
1    [invest, bot, submiss, violat, content, guid, ...
2    [sexual, identifi, wsb, young, lad, dream, soa...
3    [weight, blanket, custom, commun, award, ticke...
4    [haven, check, app, download, go, celebr, new,...
5    [post, shiiit, nikola, sub, dirt, factori, nkl...
6    [liter, tit, bot, submiss, violat, content, gu...
7    [current, situat, bot, submiss, violat, conten...
8  

In [4]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
print(bow_corpus[41])


bow_doc_80 = bow_corpus[41]
for i in range(len(bow_doc_80)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_80[i][0], 
                                               dictionary[bow_doc_80[i][0]], bow_doc_80[i][1]))

0 bot
1 give
2 good
3 growth
4 headlin
5 hell
6 massiv
7 sticki
8 best
9 buy
10 call
[(7, 2), (18, 2), (24, 1), (38, 1), (52, 1), (58, 3), (71, 1), (75, 1), (101, 1), (115, 1), (136, 1), (147, 2), (163, 1), (171, 1), (184, 1), (210, 2), (212, 1), (228, 1), (253, 1), (256, 1), (271, 1), (272, 2), (277, 1), (287, 1), (289, 1), (313, 2), (350, 1), (361, 1), (425, 2), (428, 1), (456, 1), (484, 1), (495, 1), (522, 1), (532, 1), (535, 1), (588, 1), (599, 1), (607, 1), (644, 1), (652, 1), (698, 1), (702, 1), (706, 1), (709, 1), (743, 1)]
Word 7 ("call") appears 2 time.
Word 18 ("post") appears 2 time.
Word 24 ("way") appears 1 time.
Word 38 ("guy") appears 1 time.
Word 52 ("retard") appears 1 time.
Word 58 ("think") appears 3 time.
Word 71 ("sell") appears 1 time.
Word 75 ("go") appears 1 time.
Word 101 ("hold") appears 1 time.
Word 115 ("thank") appears 1 time.
Word 136 ("open") appears 1 time.
Word 147 ("week") appears 2 time.
Word 163 ("posit") appears 1 time.
Word 171 ("hope") appears 1 t

In [5]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5318408789343217),
 (1, 0.375950112436695),
 (2, 0.15929160713668244),
 (3, 0.4329946640998139),
 (4, 0.39316170461752037),
 (5, 0.4564783359945488)]


In [11]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=8, id2word=dictionary, passes=20, iterations=400, workers=4)

# print(lda_model.print_topics(num_topics=5, num_words=5))

for idx, topic in lda_model.print_topics(num_topics=7, num_words=7):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 3 
Words: 0.024*"money" + 0.016*"tax" + 0.015*"loan" + 0.012*"work" + 0.012*"year" + 0.012*"gain" + 0.012*"congrat"
Topic: 4 
Words: 0.044*"sell" + 0.025*"call" + 0.018*"hold" + 0.018*"share" + 0.017*"option" + 0.015*"gain" + 0.014*"plug"
Topic: 2 
Words: 0.038*"steel" + 0.035*"china" + 0.020*"product" + 0.017*"compani" + 0.016*"new" + 0.014*"stock" + 0.014*"covid"
Topic: 5 
Words: 0.023*"share" + 0.013*"go" + 0.012*"sell" + 0.011*"stock" + 0.011*"think" + 0.011*"get" + 0.010*"gme"
Topic: 0 
Words: 0.028*"tesla" + 0.016*"year" + 0.016*"compani" + 0.014*"money" + 0.012*"think" + 0.012*"elon" + 0.012*"stock"
Topic: 7 
Words: 0.018*"day" + 0.017*"market" + 0.015*"close" + 0.015*"call" + 0.012*"ban" + 0.012*"work" + 0.011*"time"
Topic: 1 
Words: 0.019*"peopl" + 0.017*"trump" + 0.014*"think" + 0.012*"shit" + 0.012*"get" + 0.011*"go" + 0.010*"right"


In [12]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=8, id2word=dictionary,passes=20, iterations=400, workers=4)

for idx, topic in lda_model_tfidf.print_topics(num_topics=7, num_words=7):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 2 Word: 0.010*"funni" + 0.008*"tesla" + 0.006*"launch" + 0.004*"know" + 0.004*"move" + 0.004*"leav" + 0.001*"pass"
Topic: 6 Word: 0.008*"tank" + 0.006*"news" + 0.005*"stimulu" + 0.005*"gold" + 0.005*"repeat" + 0.005*"reddit" + 0.004*"ipo"
Topic: 4 Word: 0.009*"tsla" + 0.006*"bud" + 0.006*"pl" + 0.006*"rock" + 0.006*"rip" + 0.006*"daddi" + 0.005*"exit"
Topic: 0 Word: 0.015*"elon" + 0.015*"tesla" + 0.011*"earn" + 0.010*"ipo" + 0.009*"musk" + 0.009*"bezo" + 0.009*"papa"
Topic: 5 Word: 0.007*"sell" + 0.006*"share" + 0.006*"call" + 0.005*"stock" + 0.005*"go" + 0.005*"think" + 0.005*"hold"
Topic: 1 Word: 0.010*"vote" + 0.008*"click" + 0.007*"send" + 0.007*"messag" + 0.006*"bot" + 0.006*"remov" + 0.005*"exit"
Topic: 7 Word: 0.008*"cent" + 0.008*"away" + 0.008*"eye" + 0.007*"close" + 0.006*"confus" + 0.006*"app" + 0.006*"sound"


In [8]:
# print(processed_docs[101])

# for index, score in sorted(lda_model[bow_corpus[80]], key=lambda tup: -1*tup[1]):
#     print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))